In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import tensorflow as tf
import numpy as np
import random
import json
import pickle
import spacy
import datetime
import tflearn
import pandas as pd
from fuzzywuzzy import process
from timefhuman import timefhuman

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [11]:
words = []
classes = []
documents = []
ignore = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [12]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

17 documents
4 classes ['book_table', 'goodbye', 'greeting', 'thanks']
32 unique stemmed words ["'s", 'a', 'any', 'anyon', 'ar', 'at', 'book', 'bye', 'can', 'day', 'do', 'find', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'how', 'in', 'is', 'lat', 'me', 'pleas', 'resta', 'seat', 'see', 'tabl', 'thank', 'that', 'ther', 'you']


In [13]:
training = []
output = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [14]:
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

print(len(train_x))
print(train_x)
print(len(train_y))
print(train_y)

17
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 

In [ ]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [ ]:
model.fit(train_x, train_y, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')

In [17]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)  
  for s in sentence_words:
    for i,w in enumerate(words):
      if w == s: 
        bag[i] = 1

    return np.array(bag)

In [18]:
ERROR_THRESHOLD = 0.30

In [19]:
def classify(sentence):
  results = model.predict([bow(sentence, words)])[0]
  results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]], r[1]))
  return return_list

In [20]:
def response(sentence):
  results = classify(sentence)
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))

      results.pop(0)

In [21]:
def find_restaurant(string, restaurants):
    return process.extractOne(string, restaurants)

In [22]:
def load_data():
  
  with open('restaurants.txt') as f:
    restaurants = f.read().split('\n')
  rest_lower = []

  for restaurant in restaurants:
    rest_lower.append(restaurant.lower().replace("'", "").replace('.', '').replace('!', '').replace('-', ' ').replace('é', 'e'))
    
  return rest_lower

In [23]:
rest_choices = ['Which restaurant do you want to book in?', 'Which restraunt you prefer?', 'Which restaurant do like to book a table in?']
date_and_time_choices = ['Which date and time do you prefer?', 'Please tell me your preffered date and time.']
people_choices = ['How many people are there?', 'For how many people?']

In [24]:
rest_lower = load_data()
nlp = spacy.load("en_core_web_sm")

In [25]:
# Find restaurant from the list (with confidence)
find_restaurant('box8', rest_lower)

('box8', 100)

In [26]:
def book_table(string, restarants):
  entities_found = [False, False, False]        # Restaurant name, date and time, # of people
  entities = []
  first = [True, True, True]
  
  while not all(entities_found):
    
    # For Restaurant Name
    
    while not entities_found[0]:
      if first[0]:
        _string = string
      else:
        print(rest_choices[random.choice([x for x in range(len(rest_choices))])])
        _string = input()
      restaurant = find_restaurant(_string, restarants)
      print(restaurant)
      if restaurant != None and restaurant[1]>=60:
        entities.append(restaurant[0])
        entities_found[0] = True
      first[0] = False

        
    # For Date and Time
    # Input must be in MM/DD/YYYY with time
    # If input contains any integer, it recognizes it as time (even for # of people)

    while entities_found[1] == False:
      if first[1]:
        _string = string
      else:
        print(date_and_time_choices[random.choice([x for x in range(len(date_and_time_choices))])])
        _string = input()
      date_time = timefhuman(_string)
      print(date_time)
      if isinstance(date_time, datetime.datetime):
        date_time = date_time
      elif len(date_time) == 0:
        first[1] = False
        continue
      else:
        date_time = date_time[0]
      entities.append(date_time.strftime('%x %X'))
      entities_found[1] = True
      first[1] = False

    # For # of people

    while entities_found[2] == False:
      if first[2]:
        _string = string
      else:
        print(people_choices[random.choice([x for x in range(len(people_choices))])])
        _string = input()
      doc = nlp(_string)
      for token in doc.ents:
        if token.label_ == 'CARDINAL' or token.label_ == 'ORDINAL':
          entities.append(token.text)
          entities_found[2] = True
      first[2] = False
    
  return entities

In [28]:
while(True):
  raw_input = input()

  if raw_input == 'close':
    break
  if classify(raw_input)[0][0] == 'book_table':
    result = book_table(raw_input, rest_lower)
    print('-'*50)
    print('Table Booked')
    print('Resturant:', result[0])
    print('Date and Time:', result[1])
    print('Number of people:', result[2])
    break
  else:
    response(raw_input)

hi
Hello, thanks for visiting
book me a table
('taco bell', 51)
Which restaurant do like to book a table in?
mc donalds
('mcdonalds', 95)
[]
Which date and time do you prefer?
08/22/2020 at 8 pm
2020-08-22 20:00:00
How many people are there?
5
--------------------------------------------------
Table Booked
Resturant: mcdonalds
Date and Time: 08/22/20 20:00:00
Number of people: 5
